In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# MapBox API 호출 예시
import requests
import time
from tqdm import tqdm
from datetime import datetime
import geocoder
import os

In [2]:
# 한글 폰트 설정 (필요한 경우)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


### 1. 데이터 불러오기

In [3]:
# 엑셀 파일 경로 설정
file_path = "/Users/stella/Documents/03.proj/p_truck_delivery/data/winter_internship_prj_250122_v1.0.xlsx"  # 파일명을 실제 파일 이름으로 변경하세요.

# Excel 파일 읽기 (engine='openpyxl' 명시)
excel_data = pd.ExcelFile(file_path, engine='openpyxl')

# 모든 시트 이름 확인 (선택적으로 확인 가능)
print("Available sheets:", excel_data.sheet_names)

# 두 번째 시트의 데이터 읽기 (시트 인덱스는 0부터 시작)
df = pd.read_excel(file_path, sheet_name=1, engine='openpyxl')  # sheet_name=1은 두 번째 시트를 의미

# 데이터프레임 확인
print(df)

Available sheets: ['info', 'data_sample', 'sample2', 'Analysis']
        GpsProvider           BookingID Market/Regular   \
0     CONSENT TRACK  MVCV0000927/082021          Market   
1           VAMOSYS  VCV00014271/082021         Regular   
2     CONSENT TRACK  VCV00014382/082021         Regular   
3           VAMOSYS  VCV00014743/082021         Regular   
4           VAMOSYS  VCV00014744/082021         Regular   
...             ...                 ...             ...   
6875          JTECH        WDSBKTP42751         Regular   
6876          JTECH        WDSBKTP43203         Regular   
6877          JTECH        WDSBKTP43021         Regular   
6878          JTECH        WDSBKTP42685         Regular   
6879          JTECH        WDSBKTP42858         Regular   

              BookingID_Date  vehicle_no  \
0    2020-08-17 14:59:01.000    KA590408   
1    2020-08-27 16:22:22.827  TN30BC5917   
2    2020-08-27 17:59:24.987  TN22AR2748   
3    2020-08-28 00:48:24.503  TN28AQ0781   
4    2

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   GpsProvider                         5927 non-null   object        
 1   BookingID                           6880 non-null   object        
 2   Market/Regular                      6880 non-null   object        
 3   BookingID_Date                      6880 non-null   datetime64[ns]
 4   vehicle_no                          6880 non-null   object        
 5   Origin_Location                     6880 non-null   object        
 6   Destination_Location                6880 non-null   object        
 7   Org_lat_lon                         6880 non-null   object        
 8   Des_lat_lon                         6880 non-null   object        
 9   Data_Ping_time                      5927 non-null   datetime64[ns]
 10  Planned_ETA             

### 2. 탐색적 데이터 분석(EDA)

#### 2.1 컬럼명 정제

In [5]:
def convert_to_camel_case(column_name):
    # 특수문자를 공백으로 대체하고 공백을 기준으로 분리
    words = ''.join(c if c.isalnum() else ' ' for c in column_name).split()
    
    # 첫 번째 단어는 소문자로, 나머지 단어들은 첫 글자만 대문자로 변환
    return words[0].lower() + ''.join(word.capitalize() for word in words[1:])

In [6]:
# 데이터프레임의 모든 컬럼명을 camel case로 변환
def convert_columns_to_camel_case(df):
    # 현재 컬럼명과 변환된 컬럼명을 매핑
    column_mapping = {col: convert_to_camel_case(col) for col in df.columns}
    
    # 변경된 컬럼명 출력 (변경 전/후 확인용)
    print("컬럼명 변경 사항:")
    for old_col, new_col in column_mapping.items():
        print(f"{old_col} -> {new_col}")
    
    # 데이터프레임 컬럼명 변경
    df.rename(columns=column_mapping, inplace=True)
    
    return df

In [7]:
# 컬럼명을 camel case로 변환
df = convert_columns_to_camel_case(df)

# 결과 확인
print("\n변환된 컬럼명:")
print(df.columns.tolist())

컬럼명 변경 사항:
GpsProvider -> gpsprovider
BookingID -> bookingid
Market/Regular  -> marketRegular
BookingID_Date -> bookingidDate
vehicle_no -> vehicleNo
Origin_Location -> originLocation
Destination_Location -> destinationLocation
Org_lat_lon -> orgLatLon
Des_lat_lon -> desLatLon
Data_Ping_time -> dataPingTime
Planned_ETA -> plannedEta
Current_Location -> currentLocation
DestinationLocation -> destinationlocation
actual_eta -> actualEta
Curr_lat -> currLat
Curr_lon -> currLon
ontime -> ontime
delay -> delay
OriginLocation_Code -> originlocationCode
DestinationLocation_Code -> destinationlocationCode
trip_start_date -> tripStartDate
trip_end_date -> tripEndDate
TRANSPORTATION_DISTANCE_IN_KM -> transportationDistanceInKm
vehicleType -> vehicletype
Minimum_kms_to_be_covered_in_a_day -> minimumKmsToBeCoveredInADay
Driver_Name -> driverName
Driver_MobileNo -> driverMobileno
customerID -> customerid
customerNameCode -> customernamecode
supplierID -> supplierid
supplierNameCode -> suppliernameco

#### 2.2 시간 컬럼 정제
    - tripStartDate, tripEndDate 간의 소요시간 생성

In [8]:
# 운행시간 처리
df['tripStartDate'] = pd.to_datetime(df['tripStartDate'])
df['tripEndDate'] = pd.to_datetime(df['tripEndDate'])
df['tripDuration'] = (df['tripEndDate'] - df['tripStartDate']).dt.total_seconds() / 3600  # 시간 단위

In [9]:
# 분석에 필요한 feature 정보 추출
List = ['gpsprovider','bookingid', 'bookingidDate', 'vehicleNo', 'originLocation', 'destinationLocation', 'orgLatLon', 'desLatLon', 'plannedEta'
        , 'currentLocation', 'destinationlocation', 'actualEta', 'ontime', 'delay', 'tripStartDate', 'tripEndDate', 'tripDuration', 'transportationDistanceInKm'
        ,'minimumKmsToBeCoveredInADay', 'customerid', 'customernamecode']
# test=df[List].copy()
test=df.copy()

#### 2.3 기초분석

In [10]:
def analyze_dataframe(df):
    """
    데이터프레임 기초분석 함수
    """
    print("="*50)
    print("1. 데이터프레임 기본 정보")
    print("="*50)
    print(f"데이터 크기: {df.shape}")
    print("\n데이터 타입 정보:")
    print(df.info())
    
    print("\n"+"="*50)
    print("2. 결측치 분석")
    print("="*50)
    # 결측치 개수와 비율 계산
    missing_values = df.isnull().sum()
    missing_ratio = (df.isnull().sum() / len(df)) * 100
    
    missing_info = pd.DataFrame({
        '결측치 개수': missing_values,
        '결측치 비율(%)': missing_ratio.round(2)
    })
    print(missing_info[missing_info['결측치 개수'] > 0])  # 결측치가 있는 컬럼만 출력
    
    print("\n"+"="*50)
    print("3. 수치형 변수 기초 통계량")
    print("="*50)
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    print(df[numeric_columns].describe())
    
    print("\n"+"="*50)
    print("4. 범주형 변수 분포")
    print("="*50)
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        print(f"\n{col} 변수의 고유값 개수: {df[col].nunique()}")
        print(f"\n상위 5개 빈도:")
        print(df[col].value_counts().head())

In [11]:
analyze_dataframe(test)

1. 데이터프레임 기본 정보
데이터 크기: (6880, 33)

데이터 타입 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   gpsprovider                  5927 non-null   object        
 1   bookingid                    6880 non-null   object        
 2   marketRegular                6880 non-null   object        
 3   bookingidDate                6880 non-null   datetime64[ns]
 4   vehicleNo                    6880 non-null   object        
 5   originLocation               6880 non-null   object        
 6   destinationLocation          6880 non-null   object        
 7   orgLatLon                    6880 non-null   object        
 8   desLatLon                    6880 non-null   object        
 9   dataPingTime                 5927 non-null   datetime64[ns]
 10  plannedEta                   6880 non-null   object        
 

In [13]:
# 새로운 Unique Key 생성
def create_unique_key(df):
    # bookingid와 customerid를 문자열로 변환하고 결합
    # df['uniqueKey'] = df['gpsprovider'].astype(str)+ '_' + df['bookingid'].astype(str) + '_' + df['customerid'].astype(str)
    df['uniqueKey'] = df['bookingid'].astype(str) + '_' + df['customerid'].astype(str) + '_' + df['gpsprovider'].astype(str)
    
    # uniqueKey의 고유성 확인
    total_rows = len(df)
    unique_keys = len(df['uniqueKey'].unique())
    
    print(f"전체 행 수: {total_rows}")
    print(f"고유 키 수: {unique_keys}")
    print(f"중복 키 수: {total_rows - unique_keys}")
    
    # 중복된 키가 있다면 중복 케이스 출력
    if total_rows != unique_keys:
        print("\n중복된 키 목록:")
        duplicates = df[df['uniqueKey'].duplicated(keep=False)].sort_values('uniqueKey')
        print(duplicates[['uniqueKey', 'bookingid', 'customerid', 'gpsprovider']])
        
        # 중복 해결을 위해 순번 추가
        df['uniqueKey'] = df.groupby('uniqueKey').cumcount().astype(str) + '_' + df['uniqueKey']
    
    return df

# 함수 실행
test = create_unique_key(test)

# 결과 확인
print("\n최종 Unique Key 확인:")
print(f"전체 행 수: {len(test)}")
print(f"고유 키 수: {len(test['uniqueKey'].unique())}")

# 샘플 데이터 확인
print("\n생성된 Unique Key 샘플:")
print(test[['uniqueKey', 'bookingid', 'customerid', 'gpsprovider']].head())

전체 행 수: 6880
고유 키 수: 6880
중복 키 수: 0

최종 Unique Key 확인:
전체 행 수: 6880
고유 키 수: 6880

생성된 Unique Key 샘플:
                                     uniqueKey           bookingid  \
0  MVCV0000927/082021_ALLEXCHE45_CONSENT TRACK  MVCV0000927/082021   
1        VCV00014271/082021_DMREXCHEUX_VAMOSYS  VCV00014271/082021   
2  VCV00014382/082021_LUTGCCHE06_CONSENT TRACK  VCV00014382/082021   
3        VCV00014743/082021_DMREXCHEUX_VAMOSYS  VCV00014743/082021   
4        VCV00014744/082021_LUTGCCHE06_VAMOSYS  VCV00014744/082021   

   customerid    gpsprovider  
0  ALLEXCHE45  CONSENT TRACK  
1  DMREXCHEUX        VAMOSYS  
2  LUTGCCHE06  CONSENT TRACK  
3  DMREXCHEUX        VAMOSYS  
4  LUTGCCHE06        VAMOSYS  


In [14]:
def create_unified_delivery_status(df):
    """
    ontime('G')와 delay('R') 컬럼을 하나의 배송 상태 컬럼으로 통합
    
    Parameters:
        df: DataFrame with 'ontime' and 'delay' columns
    
    Returns:
        DataFrame with new 'delivery_status' column
    """
    # 기존 값 확인
    print("=== 기존 데이터 값 분포 ===")
    print("\nontime 컬럼 값 분포:")
    print(df['ontime'].value_counts())
    print("\ndelay 컬럼 값 분포:")
    print(df['delay'].value_counts())
    
    # 새로운 배송 상태 컬럼 생성
    df['delivery_status'] = 'UNKNOWN'  # 기본값
    
    # ontime이 'G'인 경우 'ON_TIME'으로 설정
    df.loc[df['ontime'] == 'G', 'delivery_status'] = 'ontime'
    
    # delay가 'R'인 경우 'DELAYED'로 설정
    df.loc[df['delay'] == 'R', 'delivery_status'] = 'delay'
    
    # 데이터 검증
    # ontime이 'G'인데 delay도 'R'인 경우 확인 (모순된 데이터)
    contradictions = df[(df['ontime'] == 'G') & (df['delay'] == 'R')]
    if len(contradictions) > 0:
        print("\n=== 주의: 모순된 데이터 발견 ===")
        print(f"모순된 데이터 수: {len(contradictions)}")
        print("\n모순된 데이터 샘플:")
        print(contradictions[['ontime', 'delay', 'delivery_status']].head())
    
    # 결과 확인
    print("\n=== 새로운 배송 상태 분포 ===")
    print(df['delivery_status'].value_counts())
    
    # 숫자형 컬럼도 추가 (머신러닝 모델링용)
    df['delivery_status_code'] = df['delivery_status'].map({
        'ontime': 0,
        'delay': 1,
        'UNKNOWN': -1
    })
    
    return df

In [15]:
test=create_unified_delivery_status(test)

=== 기존 데이터 값 분포 ===

ontime 컬럼 값 분포:
ontime
G    2548
Name: count, dtype: int64

delay 컬럼 값 분포:
delay
R    4342
Name: count, dtype: int64

=== 주의: 모순된 데이터 발견 ===
모순된 데이터 수: 24

모순된 데이터 샘플:
     ontime delay delivery_status
5820      G     R           delay
5821      G     R           delay
5822      G     R           delay
5823      G     R           delay
5824      G     R           delay

=== 새로운 배송 상태 분포 ===
delivery_status
delay      4342
ontime     2524
UNKNOWN      14
Name: count, dtype: int64


In [16]:
test[test['delivery_status_code']==-1]

,gpsprovider,bookingid,marketRegular,bookingidDate,vehicleNo,originLocation,destinationLocation,orgLatLon,desLatLon,dataPingTime,...,driverMobileno,customerid,customernamecode,supplierid,suppliernamecode,materialShipped,tripDuration,uniqueKey,delivery_status,delivery_status_code
1514,VINAYAKA_TVS,AEIBK2025104,Regular,2020-08-07 11:27:20,KA52A5807,"Onnalvadi, Krishnagiri, Tamil Nadu","Shive, Pune, Maharashtra","12.683589,77.859239","18.750621,73.87719",2020-08-11 14:45:25,...,9.113995e+09,NEMEXGURH5,Neel metal products ltd,67992,SRI VINAYAKA TRANSPORT,AUTO PARTS,99.127778,AEIBK2025104_NEMEXGURH5_VINAYAKA_TVS,UNKNOWN,-1
1595,VAMOSYS,AEIBK2025125,Regular,2020-08-07 15:56:38,GJ02Z0239,"Kalri, Mahesana, Gujarat","Solgam, Ahmedabad, Gujarat","23.5159,72.077925","23.338649,71.975699",2020-08-10 09:25:17,...,8.238666e+09,GTKEXGURJS,G-tekt india pvt ltd.,81143,NEW DWARKESH TRANSPORT,SHAFT,65.489444,AEIBK2025125_GTKEXGURJS_VAMOSYS,UNKNOWN,-1
2725,CONSENT TRACK,AEIBK2021746,Regular,2020-06-30 21:47:14,DL01GC5983,"Peenya Small Industries, Bangalore, Karnataka","Dhatir, Faridabad, Haryana","13.025282,77.510345","28.192852,77.249137",2020-07-04 10:25:10,...,9.813299e+09,WENEXBAN18,Wipro enterprises pvt ltd,68912,JUPITER EXPRESS CARRIER PVT. LTD,AUTO PARTS,687.829444,AEIBK2021746_WENEXBAN18_CONSENT TRACK,UNKNOWN,-1
2732,CONSENT TRACK,AEIBK2021951,Regular,2020-07-03 17:44:03,DL01GC6187,"Peenya Small Industries, Bangalore, Karnataka","Dhatir, Faridabad, Haryana","13.025282,77.510345","28.192852,77.249137",2020-07-08 09:25:09,...,8.302295e+09,WENEXBAN18,Wipro enterprises pvt ltd,68912,JUPITER EXPRESS CARRIER PVT. LTD,"NECK COMP,FUEL FILLER",137.632500,AEIBK2021951_WENEXBAN18_CONSENT TRACK,UNKNOWN,-1
2733,EKTA,AEIBK2021962,Regular,2020-07-03 22:44:53,HR47D0664,"Irungattukottai, Kanchipuram, Tamil Nadu","Dhatir, Faridabad, Haryana","13.010768,79.993135","28.192852,77.249137",2020-07-09 11:00:18,...,7.082423e+09,WENEXBAN18,Wipro enterprises pvt ltd,999,Unknown,89210M68P20,132.618611,AEIBK2021962_WENEXBAN18_EKTA,UNKNOWN,-1
2734,CONSENT TRACK,AEIBK2022212,Regular,2020-07-08 11:53:00,RJ14GF3519,"Peenya Small Industries, Bangalore, Karnataka","Mahindra World City, Jaipur, Rajasthan","13.025282,77.510345","27.033987,75.776267",2020-07-08 11:05:08,...,8.168428e+09,WENEXBAN18,Wipro enterprises pvt ltd,68912,JUPITER EXPRESS CARRIER PVT. LTD,AUTO PARTS,23.483333,AEIBK2022212_WENEXBAN18_CONSENT TRACK,UNKNOWN,-1
5379,CONSENT TRACK,AEIBK1901943,Regular,2019-10-20 16:36:54,KA01AE9163,"Mugabala, Bangalore Rural, Karnataka","Anekal, Bangalore, Karnataka","16.560192249175344,80.792293091599547","12.777874729699617,77.642275537347089",2019-10-22 10:10:15,...,NaN,LTLEXMUM40,Larsen & toubro limited,55556,A S TRANSPORTS,EMPTY LEAD COVER,41.368333,AEIBK1901943_LTLEXMUM40_CONSENT TRACK,UNKNOWN,-1
5384,MANUAL,AEIBK1901336,Regular,2019-10-15 13:08:50,TN52U1029,"Adavipalem, West Godavari, Andhra Pradesh","Parandur, Kanchipuram, Tamil Nadu","16.510615,81.75956","12.890556,79.72877",2019-10-16 06:55:01,...,NaN,LTLEXMUM40,Larsen & toubro limited,63682,MULURA LOGISTICS PRIVATE LIMITDE,GRP6A-SLEEVE PACK,188.986111,AEIBK1901336_LTLEXMUM40_MANUAL,UNKNOWN,-1
5390,MANUAL,AEIBK1901349,Regular,2019-10-15 17:49:14,AP16TH6969,"Nidamanuru, Krishna, Andhra Pradesh","Apsp Colony, Guntur, Andhra Pradesh","16.500876,80.760239","16.449872,80.532732",2019-10-16 06:50:00,...,NaN,LTLEXMUM40,Larsen & toubro limited,63916,SHRI SAI ENTERPRISES,PLASTIC SEPERATER,140.596111,AEIBK1901349_LTLEXMUM40_MANUAL,UNKNOWN,-1
5412,MANUAL,AEIBK1901351,Regular,2019-10-15 17:48:55,AP16TY1967,"Nidamanuru, Krishna, Andhra Pradesh","Apsp Colony, Guntur, Andhra Pradesh","16.500876,80.760239","16.449872,80.532732",2019-10-16 06:55:01,...,NaN,LTLEXMUM40,Larsen & toubro limited,63916,SHRI SAI ENTERPRISES,RECTIFIER ARRANGEMENT,81.568056,AEIBK1901351_LTLEXMUM40_MANUAL,UNKNOWN,-1


In [17]:
# 데이터 품질 검증 함수
def validate_delivery_status(df):
    """
    통합된 배송 상태 데이터의 품질 검증
    """
    print("=== 데이터 품질 검증 ===")
    
    # 전체 레코드 수
    total_records = len(df)
    print(f"\n전체 레코드 수: {total_records}")
    
    # 각 상태별 건수와 비율
    status_counts = df['delivery_status'].value_counts()
    status_percentages = (status_counts / total_records * 100).round(2)
    
    print("\n배송 상태별 분포:")
    for status in status_counts.index:
        print(f"{status}: {status_counts[status]} 건 ({status_percentages[status]}%)")
    
    # UNKNOWN 상태 확인
    unknown_records = df[df['delivery_status'] == 'UNKNOWN']
    if len(unknown_records) > 0:
        print("\n=== UNKNOWN 상태 데이터 샘플 ===")
        print(unknown_records[['ontime', 'delay', 'delivery_status']].head())
    
    return status_counts, status_percentages

In [18]:
validate_delivery_status(test)

=== 데이터 품질 검증 ===

전체 레코드 수: 6880

배송 상태별 분포:
delay: 4342 건 (63.11%)
ontime: 2524 건 (36.69%)
UNKNOWN: 14 건 (0.2%)

=== UNKNOWN 상태 데이터 샘플 ===
     ontime delay delivery_status
1514    NaN   NaN         UNKNOWN
1595    NaN   NaN         UNKNOWN
2725    NaN   NaN         UNKNOWN
2732    NaN   NaN         UNKNOWN
2733    NaN   NaN         UNKNOWN


(delivery_status
 delay      4342
 ontime     2524
 UNKNOWN      14
 Name: count, dtype: int64,
 delivery_status
 delay      63.11
 ontime     36.69
 UNKNOWN     0.20
 Name: count, dtype: float64)

#### Key 결합 작업하기

In [19]:
# test.to_excel('truck_unique_key_data_250131_v1.0.xlsx', index=False)

### 3. mapBox API를 이용한 위치정보 정제

In [22]:
# MapBox 액세스 토큰 설정
MAPBOX_ACCESS_TOKEN = ""

#### 3.1 위도, 경도 분리

In [18]:
# 위도, 경도 분리 함수
def split_lat_lon(coord_str):
    """
    '위도,경도' 형식의 문자열을 분리하여 위도와 경도를 반환
    """
    try:
        lat, lon = map(float, coord_str.split(','))
        return pd.Series({'latitude': lat, 'longitude': lon})
    except:
        return pd.Series({'latitude': None, 'longitude': None})

In [36]:
# orgLatLon 컬럼을 위도, 경도로 분리
test[['org_latitude', 'org_longitude']] = test['orgLatLon'].apply(split_lat_lon)

# desLatLon 컬럼을 위도, 경도로 분리
test[['des_latitude', 'des_longitude']] = test['desLatLon'].apply(split_lat_lon)

In [37]:
test.head(2)

,gpsprovider,bookingid,bookingidDate,vehicleNo,originLocation,destinationLocation,orgLatLon,desLatLon,plannedEta,currentLocation,...,customerid,customernamecode,uniqueKey,delivery_status,delivery_status_code,org_latitude,org_longitude,org_locationDetail,des_latitude,des_longitude
0,CONSENT TRACK,MVCV0000927/082021,2020-08-17 14:59:01.000,KA590408,"TVSLSL-PUZHAL-HUB,CHENNAI,TAMIL NADU","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA","13.1550,80.1960","12.7400,77.8200",2020-08-21 18:59:01,"Vaniyambadi Rd, Valayambattu, Tamil Nadu 63575...",...,ALLEXCHE45,Ashok leyland limited,CONSENT TRACK_MVCV0000927/082021_ALLEXCHE45,delay,1,13.155,80.196,"Vanagaram - Ambathur - Puzhal Road, 수라페트, 6000...",12.740,77.820
1,VAMOSYS,VCV00014271/082021,2020-08-27 16:22:22.827,TN30BC5917,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",2020-08-31 20:22:22.827000,"Unnamed Road, Oragadam Industrial Corridor, Va...",...,DMREXCHEUX,Daimler india commercial vehicles pvt lt,VAMOSYS_VCV00014271/082021_DMREXCHEUX,ontime,0,12.839,79.954,Singaperumalkoil - Sriperumbudur - Thiruvallur...,12.839,79.954


In [33]:
# MapBox API를 사용한 reverse geocoding 함수
def get_location_info(lat, lon, access_token):
    """
    MapBox API를 사용하여 위도/경도 기반 위치 정보 조회
    """
    base_url = "https://api.mapbox.com/geocoding/v5/mapbox.places"
    
    try:
        url = f"{base_url}/{lon},{lat}.json?access_token={access_token}&language=ko"
        response = requests.get(url)
        data = response.json()
        
        if data.get('features'):
            # 첫 번째 결과의 place_name 반환
            return data['features'][0]['place_name']
        return None
    except Exception as e:
        print(f"에러 발생: {e}")
        return None

In [34]:
# 위치 정보 조회 및 새 컬럼 추가
def get_all_locations(df, access_token):
    """
    데이터프레임의 모든 위치에 대한 정보 조회
    """
    locations = []
    for idx, row in df.iterrows():
        if pd.notna(row['org_latitude']) and pd.notna(row['org_longitude']):
            location = get_location_info(row['org_latitude'], row['org_longitude'], access_token)
            locations.append(location)
            
            # API 호출 제한을 위한 지연
            time.sleep(0.1)  # 100ms 대기
            
            # 진행상황 출력 (100개마다)
            if idx % 100 == 0:
                print(f"처리 중: {idx}/{len(df)}")
        else:
            locations.append(None)
    
    return locations

In [35]:
# 위치 정보 조회 실행
test['org_locationDetail'] = get_all_locations(test, MAPBOX_ACCESS_TOKEN)

# 결과 확인
print("\n위치 정보 조회 결과 샘플:")
print(test[['orgLatLon', 'org_latitude', 'org_longitude', 'org_locationDetail']].head())

처리 중: 0/6880
처리 중: 100/6880
처리 중: 200/6880
처리 중: 300/6880
처리 중: 400/6880
처리 중: 500/6880
처리 중: 600/6880
처리 중: 700/6880
처리 중: 800/6880
처리 중: 900/6880
처리 중: 1000/6880
처리 중: 1100/6880
처리 중: 1200/6880
처리 중: 1300/6880
처리 중: 1400/6880
처리 중: 1500/6880
처리 중: 1600/6880
처리 중: 1700/6880
처리 중: 1800/6880
처리 중: 1900/6880
처리 중: 2000/6880
처리 중: 2100/6880
처리 중: 2200/6880
처리 중: 2300/6880
처리 중: 2400/6880
처리 중: 2500/6880
처리 중: 2600/6880
처리 중: 2700/6880
처리 중: 2800/6880
처리 중: 2900/6880
처리 중: 3000/6880
처리 중: 3100/6880
처리 중: 3200/6880
처리 중: 3300/6880
처리 중: 3400/6880
처리 중: 3500/6880
처리 중: 3600/6880
처리 중: 3700/6880
처리 중: 3800/6880
처리 중: 3900/6880
처리 중: 4000/6880
처리 중: 4100/6880
처리 중: 4200/6880
처리 중: 4300/6880
처리 중: 4400/6880
처리 중: 4500/6880
처리 중: 4600/6880
처리 중: 4700/6880
처리 중: 4800/6880
처리 중: 4900/6880
처리 중: 5000/6880
처리 중: 5100/6880
처리 중: 5200/6880
처리 중: 5300/6880
처리 중: 5400/6880
처리 중: 5500/6880
처리 중: 5600/6880
처리 중: 5700/6880
처리 중: 5800/6880
처리 중: 5900/6880
처리 중: 6000/6880
처리 중: 6100/6880
처리 중: 6200/6880
처리 중

In [38]:
# 위치 정보 조회 실행
test['des_locationDetail'] = get_all_locations(test, MAPBOX_ACCESS_TOKEN)

# 결과 확인
print("\n위치 정보 조회 결과 샘플:")
print(test[['desLatLon', 'des_latitude', 'des_longitude', 'des_locationDetail']].head())

처리 중: 0/6880
처리 중: 100/6880
처리 중: 200/6880
처리 중: 300/6880
처리 중: 400/6880
처리 중: 500/6880
처리 중: 600/6880
처리 중: 700/6880
처리 중: 800/6880
처리 중: 900/6880
처리 중: 1000/6880
처리 중: 1100/6880
처리 중: 1200/6880
처리 중: 1300/6880
처리 중: 1400/6880
처리 중: 1500/6880
처리 중: 1600/6880
처리 중: 1700/6880
처리 중: 1800/6880
처리 중: 1900/6880
처리 중: 2000/6880
처리 중: 2100/6880
처리 중: 2200/6880
처리 중: 2300/6880
처리 중: 2400/6880
처리 중: 2500/6880
처리 중: 2600/6880
처리 중: 2700/6880
처리 중: 2800/6880
처리 중: 2900/6880
처리 중: 3000/6880
처리 중: 3100/6880
처리 중: 3200/6880
처리 중: 3300/6880
처리 중: 3400/6880
처리 중: 3500/6880
처리 중: 3600/6880
처리 중: 3700/6880
처리 중: 3800/6880
처리 중: 3900/6880
처리 중: 4000/6880
처리 중: 4100/6880
처리 중: 4200/6880
처리 중: 4300/6880
처리 중: 4400/6880
처리 중: 4500/6880
처리 중: 4600/6880
처리 중: 4700/6880
처리 중: 4800/6880
처리 중: 4900/6880
처리 중: 5000/6880
처리 중: 5100/6880
처리 중: 5200/6880
처리 중: 5300/6880
처리 중: 5400/6880
처리 중: 5500/6880
처리 중: 5600/6880
처리 중: 5700/6880
처리 중: 5800/6880
처리 중: 5900/6880
처리 중: 6000/6880
처리 중: 6100/6880
처리 중: 6200/6880
처리 중

In [40]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   gpsprovider                  5927 non-null   object        
 1   bookingid                    6880 non-null   object        
 2   bookingidDate                6880 non-null   datetime64[ns]
 3   vehicleNo                    6880 non-null   object        
 4   originLocation               6880 non-null   object        
 5   destinationLocation          6880 non-null   object        
 6   orgLatLon                    6880 non-null   object        
 7   desLatLon                    6880 non-null   object        
 8   plannedEta                   6880 non-null   object        
 9   currentLocation              5916 non-null   object        
 10  destinationlocation          6880 non-null   object        
 11  actualEta                    6843 non-null 

In [47]:
col_lst=['gpsprovider', 'bookingid', 'bookingidDate', 'vehicleNo', 'originLocation', 'destinationLocation', 'orgLatLon', 'desLatLon'
         , 'org_latitude', 'org_longitude', 'org_locationDetail', 'des_latitude', 'des_longitude', 'des_locationDetail'
         , 'plannedEta', 'currentLocation', 'destinationlocation', 'actualEta', 'tripStartDate', 'tripEndDate', 'tripDuration'
         , 'transportationDistanceInKm', 'minimumKmsToBeCoveredInADay', 'ontime', 'delay']

test2 = test[col_lst].copy()

In [48]:
test2.head(2)

,gpsprovider,bookingid,bookingidDate,vehicleNo,originLocation,destinationLocation,orgLatLon,desLatLon,org_latitude,org_longitude,...,currentLocation,destinationlocation,actualEta,tripStartDate,tripEndDate,tripDuration,transportationDistanceInKm,minimumKmsToBeCoveredInADay,ontime,delay
0,CONSENT TRACK,MVCV0000927/082021,2020-08-17 14:59:01.000,KA590408,"TVSLSL-PUZHAL-HUB,CHENNAI,TAMIL NADU","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA","13.1550,80.1960","12.7400,77.8200",13.155,80.196,...,"Vaniyambadi Rd, Valayambattu, Tamil Nadu 63575...","ASHOK LEYLAND PLANT 1- HOSUR,HOSUR,KARNATAKA",2020-08-28 14:38:04.447000,2020-08-17 14:59:01,NaT,NaN,320.0,NaN,NaN,R
1,VAMOSYS,VCV00014271/082021,2020-08-27 16:22:22.827,TN30BC5917,"DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...","12.8390,79.9540","12.8390,79.9540",12.839,79.954,...,"Unnamed Road, Oragadam Industrial Corridor, Va...","DAIMLER INDIA COMMERCIAL VEHICLES,KANCHIPURAM,...",2020-08-28 12:46:17.007000,2020-08-27 16:21:52,NaT,NaN,103.0,NaN,G,NaN


#### 3.2 org_lonlat과 des_lonlat 컬럼 간의 거리 계산

In [49]:
def get_route_info(org_lon, org_lat, des_lon, des_lat, access_token):
    """
    MapBox Directions API를 사용하여 두 지점 간의 경로 정보를 조회
    
    Parameters:
    - org_lon, org_lat: 출발지 좌표
    - des_lon, des_lat: 도착지 좌표
    - access_token: MapBox API 토큰
    
    Returns:
    - 거리(km), 예상 소요시간(분), 경로 좌표 목록
    """
    base_url = "https://api.mapbox.com/directions/v5/mapbox/driving"
    
    try:
        # API 엔드포인트 구성
        coordinates = f"{org_lon},{org_lat};{des_lon},{des_lat}"
        url = f"{base_url}/{coordinates}"
        
        # 파라미터 설정
        params = {
            'access_token': access_token,
            'geometries': 'geojson',
            'overview': 'full',
            'steps': 'true'
        }
        
        # API 요청
        response = requests.get(url, params=params)
        data = response.json()
        
        if data.get('routes'):
            route = data['routes'][0]
            distance_km = round(route['distance'] / 1000, 2)  # 미터를 킬로미터로 변환
            duration_min = round(route['duration'] / 60, 2)   # 초를 분으로 변환
            geometry = route['geometry']['coordinates']       # 경로 좌표
            
            return {
                'distance_km': distance_km,
                'duration_min': duration_min,
                'geometry': geometry
            }
        return None
    
    except Exception as e:
        print(f"에러 발생: {e}")
        return None

In [50]:
def calculate_routes_batch(df, access_token, batch_size=100):
    """
    데이터프레임의 모든 출발지-도착지 쌍에 대한 경로 정보를 배치로 처리
    
    Parameters:
    - df: 데이터프레임
    - access_token: MapBox API 토큰
    - batch_size: 한 번에 처리할 데이터 수
    """
    distances = []
    durations = []
    geometries = []
    
    total_rows = len(df)
    
    for i in range(0, total_rows, batch_size):
        batch = df.iloc[i:min(i+batch_size, total_rows)]
        
        print(f"배치 처리 중: {i+1}-{min(i+batch_size, total_rows)} / {total_rows}")
        
        for _, row in batch.iterrows():
            if pd.notna(row['org_longitude']) and pd.notna(row['des_longitude']):
                route_info = get_route_info(
                    row['org_longitude'], row['org_latitude'],
                    row['des_longitude'], row['des_latitude'],
                    access_token
                )
                
                if route_info:
                    distances.append(route_info['distance_km'])
                    durations.append(route_info['duration_min'])
                    geometries.append(route_info['geometry'])
                else:
                    distances.append(None)
                    durations.append(None)
                    geometries.append(None)
            else:
                distances.append(None)
                durations.append(None)
                geometries.append(None)
            
            time.sleep(0.1)  # API 호출 제한 준수
    
    return distances, durations, geometries

In [51]:
# 경로 정보 계산 실행
distances, durations, geometries = calculate_routes_batch(test2, MAPBOX_ACCESS_TOKEN)

# 결과를 데이터프레임에 추가
test2['calculated_distance_km'] = distances
test2['estimated_duration_min'] = durations
test2['route_geometry'] = geometries

# 결과 확인
print("\n경로 계산 결과 샘플:")
print(test2[['orgLatLon', 'desLatLon', 'calculated_distance_km', 'estimated_duration_min']].head())

# 실제 운송 거리와 계산된 거리 비교 (선택적)
if 'transportationDistanceInKm' in test2.columns:
    test2['distance_difference'] = abs(test2['transportationDistanceInKm'] - test2['calculated_distance_km'])
    
    print("\n실제 거리와 계산된 거리 차이 통계:")
    print(test2['distance_difference'].describe())

배치 처리 중: 1-100 / 6880
배치 처리 중: 101-200 / 6880
배치 처리 중: 201-300 / 6880
배치 처리 중: 301-400 / 6880
배치 처리 중: 401-500 / 6880
배치 처리 중: 501-600 / 6880
배치 처리 중: 601-700 / 6880
배치 처리 중: 701-800 / 6880
배치 처리 중: 801-900 / 6880
배치 처리 중: 901-1000 / 6880
배치 처리 중: 1001-1100 / 6880
배치 처리 중: 1101-1200 / 6880
배치 처리 중: 1201-1300 / 6880
배치 처리 중: 1301-1400 / 6880
배치 처리 중: 1401-1500 / 6880
배치 처리 중: 1501-1600 / 6880
배치 처리 중: 1601-1700 / 6880
배치 처리 중: 1701-1800 / 6880
배치 처리 중: 1801-1900 / 6880
배치 처리 중: 1901-2000 / 6880
배치 처리 중: 2001-2100 / 6880
배치 처리 중: 2101-2200 / 6880
배치 처리 중: 2201-2300 / 6880
배치 처리 중: 2301-2400 / 6880
배치 처리 중: 2401-2500 / 6880
배치 처리 중: 2501-2600 / 6880
배치 처리 중: 2601-2700 / 6880
배치 처리 중: 2701-2800 / 6880
배치 처리 중: 2801-2900 / 6880
배치 처리 중: 2901-3000 / 6880
배치 처리 중: 3001-3100 / 6880
배치 처리 중: 3101-3200 / 6880
배치 처리 중: 3201-3300 / 6880
배치 처리 중: 3301-3400 / 6880
배치 처리 중: 3401-3500 / 6880
배치 처리 중: 3501-3600 / 6880
배치 처리 중: 3601-3700 / 6880
배치 처리 중: 3701-3800 / 6880
배치 처리 중: 3801-3900 / 6880
배치 처리 중

In [53]:
test2.columns

Index(['gpsprovider', 'bookingid', 'bookingidDate', 'vehicleNo',
       'originLocation', 'destinationLocation', 'orgLatLon', 'desLatLon',
       'org_latitude', 'org_longitude', 'org_locationDetail', 'des_latitude',
       'des_longitude', 'des_locationDetail', 'plannedEta', 'currentLocation',
       'destinationlocation', 'actualEta', 'tripStartDate', 'tripEndDate',
       'tripDuration', 'transportationDistanceInKm',
       'minimumKmsToBeCoveredInADay', 'ontime', 'delay',
       'calculated_distance_km', 'estimated_duration_min', 'route_geometry',
       'distance_difference'],
      dtype='object')

In [58]:
col_lst2=['gpsprovider', 'bookingid', 'bookingidDate', 'vehicleNo', 'originLocation', 'destinationLocation', 'orgLatLon', 'desLatLon'
         , 'org_latitude', 'org_longitude', 'org_locationDetail', 'des_latitude', 'des_longitude', 'des_locationDetail'
         , 'plannedEta', 'currentLocation', 'destinationlocation', 'actualEta', 'tripStartDate', 'tripEndDate', 'tripDuration'
         , 'transportationDistanceInKm', 'minimumKmsToBeCoveredInADay'
         , 'calculated_distance_km', 'estimated_duration_min'
        #  , 'route_geometry'
         , 'distance_difference'
         , 'ontime', 'delay']

test3 = test2[col_lst2].copy()

In [62]:
test3.to_excel('winter_internship_prj_250122_v2.0.xlsx', index=False)

In [60]:
# 1. 메모리 사용량 확인
def get_size(df):
    """
    데이터프레임의 메모리 사용량을 확인하는 함수
    """
    # 메모리 사용량을 MB 단위로 변환
    memory_usage = df.memory_usage(deep=True).sum() / 1024 / 1024
    print(f"데이터프레임 메모리 사용량: {memory_usage:.2f} MB")
    
    # 컬럼별 메모리 사용량
    print("\n컬럼별 메모리 사용량:")
    for col in df.columns:
        col_memory = df[col].memory_usage(deep=True) / 1024 / 1024
        print(f"{col}: {col_memory:.2f} MB")

# 메모리 사용량 확인
get_size(test3)

데이터프레임 메모리 사용량: 9.72 MB

컬럼별 메모리 사용량:
gpsprovider: 0.41 MB
bookingid: 0.46 MB
bookingidDate: 0.05 MB
vehicleNo: 0.44 MB
originLocation: 0.59 MB
destinationLocation: 0.60 MB
orgLatLon: 0.54 MB
desLatLon: 0.52 MB
org_latitude: 0.05 MB
org_longitude: 0.05 MB
org_locationDetail: 1.41 MB
des_latitude: 0.05 MB
des_longitude: 0.05 MB
des_locationDetail: 1.41 MB
plannedEta: 0.37 MB
currentLocation: 0.73 MB
destinationlocation: 0.60 MB
actualEta: 0.37 MB
tripStartDate: 0.05 MB
tripEndDate: 0.05 MB
tripDuration: 0.05 MB
transportationDistanceInKm: 0.05 MB
minimumKmsToBeCoveredInADay: 0.05 MB
calculated_distance_km: 0.05 MB
estimated_duration_min: 0.05 MB
distance_difference: 0.05 MB
ontime: 0.27 MB
delay: 0.32 MB


### Current Location, Destination Location 거리 추출

#### data load (데이터 불러오기)

In [2]:
# 엑셀 파일 경로 설정
file_path = "/Users/stella/Documents/03.proj/p_truck_delivery/data/delivery_data_sample_250127_v2.0.xlsx"  # 파일명을 실제 파일 이름으로 변경하세요.

# Excel 파일 읽기 (engine='openpyxl' 명시)
excel_data = pd.ExcelFile(file_path, engine='openpyxl')

# 모든 시트 이름 확인 (선택적으로 확인 가능)
print("Available sheets:", excel_data.sheet_names)

# 두 번째 시트의 데이터 읽기 (시트 인덱스는 0부터 시작)
df = pd.read_excel(file_path, sheet_name=1, engine='openpyxl', usecols='B:AC')  # sheet_name=1은 두 번째 시트를 의미

# 데이터프레임 확인
print(df)

Available sheets: ['info', 'data']
        gpsprovider           bookingid           bookingidDate   vehicleNo  \
0     CONSENT TRACK  MVCV0000927/082021 2020-08-17 14:59:01.000    KA590408   
1           VAMOSYS  VCV00014271/082021 2020-08-27 16:22:22.827  TN30BC5917   
2     CONSENT TRACK  VCV00014382/082021 2020-08-27 17:59:25.000  TN22AR2748   
3           VAMOSYS  VCV00014743/082021 2020-08-28 00:48:24.503  TN28AQ0781   
4           VAMOSYS  VCV00014744/082021 2020-08-28 01:23:19.243   TN68F1722   
...             ...                 ...                     ...         ...   
6875          JTECH        WDSBKTP42751 2019-03-27 17:25:33.000    KA219502   
6876          JTECH        WDSBKTP43203 2019-03-31 15:02:34.000  KA01AE9163   
6877          JTECH        WDSBKTP43021 2019-03-29 18:56:26.000  KA01AE9163   
6878          JTECH        WDSBKTP42685 2019-03-27 08:29:45.000   KA21A3643   
6879          JTECH        WDSBKTP42858 2019-03-28 17:55:17.000   KA51D1317   

                

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   gpsprovider                 5927 non-null   object        
 1   bookingid                   6880 non-null   object        
 2   bookingidDate               6880 non-null   datetime64[ns]
 3   vehicleNo                   6880 non-null   object        
 4   originLocation              6880 non-null   object        
 5   destinationLocation         6880 non-null   object        
 6   orgLatLon                   6880 non-null   object        
 7   desLatLon                   6880 non-null   object        
 8   org_latitude                6880 non-null   float64       
 9   org_longitude               6880 non-null   float64       
 10  org_locationDetail          6880 non-null   object        
 11  des_latitude                6880 non-null   float64     

In [ ]:
key=""
g = geocoder.mapbox('Vaniyambadi Rd, Valayambattu, Tamil Nadu 635752, India', key=key)
result=g.json

In [20]:
result

{'address': 'Vaniyambadi, Tamil Nadu, India',
 'bbox': {'northeast': [12.774211, 78.832381],
  'southwest': [12.447587, 78.489152]},
 'confidence': 1,
 'country': 'India',
 'lat': 12.678486,
 'lng': 78.62049,
 'ok': True,
 'quality': 0.670635,
 'raw': {'id': 'place.46671979',
  'type': 'Feature',
  'place_type': ['place'],
  'relevance': 0.670635,
  'properties': {'mapbox_id': 'dXJuOm1ieHBsYzpBc2dvYXc',
   'wikidata': 'Q282212'},
  'text': 'Vaniyambadi',
  'place_name': 'Vaniyambadi, Tamil Nadu, India',
  'bbox': [78.489152, 12.447587, 78.832381, 12.774211],
  'center': [78.62049, 12.678486],
  'geometry': {'type': 'Point', 'coordinates': [78.62049, 12.678486]},
  'context': [{'id': 'district.5588587',
    'mapbox_id': 'dXJuOm1ieHBsYzpWVVpy',
    'wikidata': 'Q66659621',
    'text': 'Tirupathur'},
   {'id': 'region.173163',
    'mapbox_id': 'dXJuOm1ieHBsYzpBcVJy',
    'wikidata': 'Q1445',
    'short_code': 'IN-TN',
    'text': 'Tamil Nadu'},
   {'id': 'country.8811',
    'mapbox_id': '

In [19]:
result.get('place_name')


In [5]:
def process_locations(df):
    """
    데이터프레임의 주소들에 대해 위도와 경도를 추출
    """
    # 결과를 저장할 새 컬럼 생성
    # df['current_latitude'] = []
    # df['current_longitude'] = []
    df_cur_lat=[]
    df_cur_lng=[]
    
    api_key=""   
    
    for idx, row in tqdm(df.iterrows()):
        if pd.notna(row['currentLocation']):
            # geocoder로 위치 정보 요청
            g = geocoder.mapbox(row['currentLocation'], key=api_key)
            result = g.json
    
            # status가 'OK'인 경우에만 좌표 저장
            if result and result.get('status') == 'OK':
                # df['current_latitude'].append(result.get('lat'))
                # df['current_longitude'].append(result.get('lng'))
                df_cur_lat.append(result.get('lat'))
                df_cur_lng.append(result.get('lng'))    

            
            else:
                # df['current_latitude'].append(None)
                # df['current_longitude'].append(None)
                df_cur_lat.append(None)
                df_cur_lng.append(None)
                
                
    return df_cur_lat, df_cur_lng

In [ ]:
df['current_latitude'], df['current_longitude'] = process_locations(df)